In [25]:
import vtk
import numpy as np
import ipywidgets as widgets
import plotly.graph_objs as go
from vtk import *

In [26]:
#creating histogram
def create_histogram(filtered_scalar_data):
    layout = go.Layout(
    #title='Histogram',
    xaxis=dict(title='Vortex Scalar Values'),
    yaxis=dict(title='Frequency'),
    )
    return go.FigureWidget(data=[go.Histogram(x=filtered_scalar_data, nbinsx=43,marker=dict(color='blue'),opacity=0.6)], layout=layout)


In [27]:
# creating slider
def create_slider(mini,maxi):
    return widgets.FloatSlider(value=0.0, min=mini, max=maxi, description='Isovalue',continuous_update=False,step=0.01)


In [28]:
#isosurface update function
def isosurface_update(change):
    x = change.new
    fig.data[0].update(isomin=x,isomax=x)
    
#histogram update function
def histogram_update(change):
    x = change.new
    val = []
    for i in values:
        if i >= x-0.25:
            if i<= x+0.25:
                val.append(i)
        else:
            pass
    histogram.data[0].x = val
    
#histogram reset function
def histogram_reset(reset_button):
    slider.value=0.0
    histogram.data[0].x = values

#isosurface reset function    
def isosurface_reset(reset_button):
    x = 0.0
    fig.data[0].update(isomin=x,isomax=x)


In [29]:
#creating reader
reader = vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')
reader.Update()



data = reader.GetOutput()
#finding no. of dimensions
num_dim = data.GetDimensions()
num_points = num_dim[0] * num_dim[1] * num_dim[2]
data_array = data.GetPointData().GetArray('ImageFile')
values=[]
coordinates=[]
for pid in range(num_points):
    values.append(data_array.GetTuple1(pid))
    coordinates.append(data.GetPoint(pid))
    
surface = vtkGeometryFilter()
surface.SetInputData(data)
surface.Update()
image_data = surface.GetOutput()
Range = data.GetScalarRange()
#print(Range)
x=[]
y=[]
z=[]
for i in coordinates:
    x.append(i[0])
    y.append(i[1])
    z.append(i[2])



In [30]:



#point_data,scalars,values=get_val(dims)


def create_isosurface(values,x,y,z):
    data=go.Isosurface(
    x=x,
    y=y,
    z=z,
    value=values,
    isomin=0.0,
    isomax=0.0,
    cauto = False,
    cmin = min(values),
    cmax = max(values),
    surface_count=1,
    colorscale='plasma',
    caps=dict(x_show=False, y_show=False),
    opacity=1,
    colorbar=dict(thickness=20, ticklen=4),
    showscale=False,
    )
    # return go.FigureWidget(data)
    isosurface =  go.FigureWidget(data)
    isosurface.update_scenes(
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False),
        zaxis=dict(showticklabels=False),
    )
    return isosurface


fig = create_isosurface(values,x,y,z)




In [31]:
# Extract the scalar data for the vertices from the 3D dataset
output = reader.GetOutput()
point_data = output.GetPointData()
scalar_data = point_data.GetScalars()

x_min = np.min(values)
x_max = np.max(values) 

#finding filtered scalar data 
filtered_scalar_data=[]
for i in range(scalar_data.GetNumberOfTuples()):
    res=scalar_data.GetValue(i)
    if x_min <= res  and res <= x_max:
        filtered_scalar_data.append(res)
        

histogram=create_histogram(filtered_scalar_data)


In [32]:

slider=create_slider(Range[0],Range[1])
reset_button = widgets.Button(description='Reset')

#creating slider    
slider.observe(isosurface_update,names='value')
slider.observe(histogram_update,names='value')
reset_button.on_click(histogram_reset)
reset_button.on_click(isosurface_reset)

# creating widgets
widget1 = widgets.HBox([slider,reset_button])
widget2 = widgets.HBox([fig,histogram])
widget3 = widgets.VBox([widget1,widget2])
display(widget3)
